# Don't Meet Your Heroes, Clone Them (With AI)

Check out the edu-tainment video on YouTube

Check out the video walkthrough for this code

Let's try our best to recreate a person!

**Steps we'll do today:**

0. Get full episode list
1. Get transcripts for ~480 episodes (via Deepgram.com)
2. Get backstory points from random sample of podcasts
3. Identify memories chunks
4. Select good tone examples
6. Prompt it all together
7. Hook up voice in, voice out

My take on parts of a personality
* **Backstory** - Demographics about someone static. Think of this as the config file for a person 
* **Memories** - Specific stories. Might just take these raw, tbd. This could be classification for a paragraph and then selected as relevant to the topic at hand
* **Tone Style & Examples** - Exact examples of someone talking - This should be chunked and embedded

**Note:** I'm happy with backstory and memories, but tone is still difficult. It's an open exploration and I'd love to learn methods you come up with

## Step 1. Get Episode List

Below is the code to scrape off the [MFM Website](https://www.mfmpod.com/) or else you can load the list I already pulled at the bottom

In [1]:
import os
import requests
import time
import pandas as pd
import openai
import json
from deepgram import Deepgram
from enum import Enum
from IPython.display import Audio
from bs4 import BeautifulSoup
from pydantic import BaseModel, Field
from typing import Optional, List
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import Chroma

load_dotenv(dotenv_path='../.env') # Replace this with the path do your .env file if you're going that route
openai.api_key = os.getenv("OPENAI_API_KEY", "YourAPIKey")

/Users/gregorykamradt/opt/anaconda3/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.19) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


What better place to get a list of episodes than the MFM site itself?

Either run the function below or...load the list that I already pulled (recommended)

In [6]:
def get_episode_list_from_mfm(pages=1):
    # At the time of this pull there were 22 pages on the website
    links = []

    for i in range(1, pages+1):
#         print (i) # If you want to track progress
        r = requests.get(f"https://www.mfmpod.com/episodes/?page={i}")

        soup = BeautifulSoup(r.text, 'html.parser')

        for link in soup.findAll("h3", {"class" : "heading heading-3 strong-600 mb-0"}):
            links.append(link.find("a")['href'])
    return links

## Uncomment the list below if you wanted to try and pull them yourself. Increase the page number
episode_list = get_episode_list_from_mfm(1)
df = pd.DataFrame(episode_list, columns=['paths'])

The list below was pulled in June '23 so episodes after that won't be shown

In [10]:
df = pd.read_csv('MFMEpisodeList.csv')
df.head()

,title,description,mega_link,created_at
0,#1 - From making $76k at Microsoft to selling ...,"We talk with tech founder & investor Suleman ""...",https://dcs.megaphone.fm/HS3627187244.mp3,2019-06-23T02:46:00Z
1,#2 with Ramon Van Meer - Blogging His Way To $...,Ramon Van Meer (@ramonvanmeer) has never seen ...,https://dcs.megaphone.fm/HS5786568093.mp3,2019-07-02T04:00:00Z
2,#3 - Making Millions off an Email Newsletter?!...,Sam Parr (@theSamParr) is the founder of The H...,https://dcs.megaphone.fm/HS3526514479.mp3,2019-07-09T21:44:00Z
3,#4 - How Product Hunt's Ryan Hoover Built A $2...,Who knew a product about other products could ...,https://dcs.megaphone.fm/HS8110626829.mp3,2019-07-16T20:00:00Z
4,"#5 - The First Viral Website: ""Hot or Not""",James Hong (@jhong) is the creator of Hot or N...,https://dcs.megaphone.fm/HS8386697001.mp3,2019-07-23T20:00:00Z


## Step 2. Get Transcripts From Deepgram

Get access to [Deepgram](https://tinyurl.com/dgjupyter) ($200 free credits to start off)

Now that we have the audio links (under the mega_link) column, we can pass these to [Deepgram](https://tinyurl.com/dgjupyter) to scrape

Here's the code for a single episode pull. (Skip down a few cells in case you want to use the 10 transcripts I've [already done](#transcript_parsing))

In [11]:
def get_transcript(audio_url, dg_client):
    print (f"Starting: {audio_url}")
    source = {'url': audio_url}
    
    # If no model is provided in the params you'll default to their basic quick one 'Nova'
    response = dg_client.transcription.sync_prerecorded(source, {
        'punctuate': True,
        'diarize' : True,
        'utterances': True,
#         'model' : 'whisper-large' # Large is the most accurate, but slowest
#         'model' : 'whisper-tiny' # Tiny is quicker
    })
    
    print (f"Finished: {audio_url}")
    
    return response

In [12]:
## Uncomment this one if you want to run a sample
dg = Deepgram(os.getenv("DEEPGRAM_API_KEY"))
sample_episode_transcript = get_transcript(df.loc[0,'mega_link'], dg)

Starting: https://dcs.megaphone.fm/HS3627187244.mp3
Finished: https://dcs.megaphone.fm/HS3627187244.mp3


Great, that is how you do one, if you want to do the rest you can simply loop through them

In [ ]:
# transcripts = []
# for ep in df['mega_link'].values:
#     transcripts.append(get_transcript(ep, dg))

Note: The function below will help you clean up your transcripts from the raw deepgram output. By default your speakers will be 'Speaker_1'. However you can do a speaker map (like below) to put in real names

```
speaker_map = {
    0 : "Shaan",
    1 : "Shaan",
    2 : "Sam",
}
```

In [ ]:
def process_transcript(transcript, speaker_map):
    """Function to process transcript."""
    processed_transcript = []
    current_speaker_id = transcript[0]['words'][0]['speaker']  # Start with the speaker of the first word
    current_speaker_name = speaker_map.get(current_speaker_id, f"Speaker_{current_speaker_id}")
    current_sentence = []
    current_start_time = transcript[0]['start']  # Start time of the first sentence\

    for entry in transcript:
        for word in entry['words']:
            speaker_id = word['speaker']

            # If the speaker has changed, append the current sentence to the transcript
            # and start a new sentence with the new speaker
            if speaker_id != current_speaker_id:
                processed_transcript.append(f"{convert_seconds_to_hms(current_start_time)} {current_speaker_name}: {' '.join(current_sentence)}")
                current_sentence = []
                current_speaker_id = speaker_id
                current_speaker_name = speaker_map.get(current_speaker_id, f"Speaker_{current_speaker_id}")
                current_start_time = word['start']  # Update the start time to the start of the new sentence


            # Add the word to the current sentence
            current_sentence.append(word['punctuated_word'])

    # Add the last sentence to the transcript
    processed_transcript.append(f"{convert_seconds_to_hms(current_start_time)} {current_speaker_name}: {' '.join(current_sentence)}")

    return '\n\n'.join(processed_transcript)

However, a loop will take forever depending on your model and number of episodes. I've put a sample 10 transcripts below. Note: They've already been pre-processed and cleaned up. You'll need to do run the "process_transcript" function above with a speaker map to clean it up

<a id='transcript_parsing'></a>

In [14]:
# First go get the transcript file names
transcript_files = os.listdir("transcripts/")

In [15]:
# Then run through them
transcripts = []

for transcript_file in transcript_files:
    # Skipping any ineligible files
    if transcript_file[-3:] != 'txt':
        continue
        
    with open(f'transcripts/{transcript_file}', 'r') as file:
        transcripts.append(file.read())

In [16]:
print (f"We now have {len(transcripts)} transcripts to work with")

We now have 10 transcripts to work with


## Step 3. Get attributes and backstory from transcripts

Now that we have our transcripts. Let's go through each one and pull out relevant details that Shaan says about himself. This will serve as our background config document.

We'll first start off by making our data structures via the pydantic method. I like this better than messing around with raw json schemas because it's cleaner

In [19]:
class DemographicAttribute(BaseModel):
    """A single attribute or demographic about a person"""
    attribute_name: str = Field(...,description="The title or name of an attribute about a \
                                                  person, such as 'occupation', 'location', etc.")
        
    attribute_value: str = Field(...,  description="The value or detail of the attribute,\
                                                    such as 'Engineer', 'New York', etc.")
        
class Person(BaseModel):
    """Information about a person"""
    demographics: Optional[List[DemographicAttribute]]

Then in order to get the actual background attributes, let's make a function that we can pass transcripts to and extract what we need. Feel free to customize the prompt based on the person you're trying to get more information about

In [21]:
def get_background_attributes(name, transcript):
    system_prompt = f"""
    You are a helpful AI bot that is preparing a briefing for an executive at a fortune 500 company about {name}
    Your goal is to prepare a report on the attributes about a person so someone else can quickly ramp up to him
    
    You will be given a piece of text which may or may not have relevant information.
    Remember, only return demographics about {name}
    Do not include anything the person is explaining, only attributes abou them.
    Pretend you're filling out a government form about this person, each field will need a header and value

    Examples of interesting information: Age, Marital status, kids, location of living, work, foods, etc.
    You don't have to include the examples, only list items that are present. If you don't know an attribute, don't say it
    """

    response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
           {"role": "system", "content": system_prompt},
           {"role": "user", "content": transcript}
        ],
        functions=[
            {
              "name": "extract_information_about_a_person",
              "description": "Get information about a person in a list of attributes",
              "parameters": Person.schema()
            }
        ],
        function_call={"name": "extract_information_about_a_person"}
    )

    output = json.loads(response.choices[0]["message"]["function_call"]["arguments"])
    return output

Then let's loop through each transcript that we have and pull out the attributes

In [22]:
def process_transcripts_for_background(transcripts):
    # The holder for our future attributes
    attributes_list = []

    # Let's run through each transcript
    for i, ep in enumerate(transcripts):

        # Logging
        print (f"Episode: {i+1}\n")
        
        # Open up your transcript that you have locally
        with open(f"../transcripts/{ep}", "r") as file:
            transcript = file.read()

        # There are multiple speakers in our transcripts but we only want Shaan's lines, making a holder
        person_lines = []
        
        # utter = utterance. Run through each one and see if Shaan is the speaker
        for utter in transcript.split("\n\n"):
            if utter[8:13] == 'Shaan':
                person_lines.append(utter)

        # Then with Shaan's lines, go get any attributes that were said.
        # Throwing in a try/except to account for any openai timeout errors
        for i in range(0, len(person_lines), 15):
            
            attempts = 0
            while attempts < 5: # You can adjust the number of retries
                try:
                    attributes = get_background_attributes("Shaan", "\n".join(person_lines[i:i+15]))
                    for att in attributes['demographics']:
                        print(f"{att['attribute_name']}: {att['attribute_value']}")
                    attributes_list.extend(attributes['demographics'])
                    break # If successful, break out of the while loop
                except Exception as e:
                    print(e)
                    time.sleep(10) # Wait for 10 seconds (you can adjust this duration)
                    attempts += 1
            else:
                print("Failed to retrieve attributes after multiple attempts.")
                
        return attributes_list

You can run through this process to get the attributes, or jump to the cell with the text file below to use the file that I already created. Remove the `[:1]` if you want to do the full 10 files.

Uncomment and run the cell below if you'd like to get the attributes yourself, if not then head down to where we load it [locally](#background_attributes)

In [23]:
# attributes = process_transcripts_for_background(transcript_files[:1])

Once you get your attributes, you may have duplicates which we need to clean up. For example 'Name: Shaan' appears many times since the model is constantly returning his name. There is likely optimization to do in the prompt for this but that is for another time ;)

In [24]:
def concatenate_attributes(attributes):
    """
    Concatenates attributes based on their name.

    :param attributes: List of dictionaries with 'attribute_name' and 'attribute_value' keys
    :return: Dictionary with concatenated attributes
    """
    concatenated_attributes = {}

    for attribute in attributes:
        if attribute['attribute_name'].capitalize() in concatenated_attributes:
            concatenated_attributes[attribute['attribute_name'].capitalize()] += ', ' + attribute['attribute_value']
        else:
            concatenated_attributes[attribute['attribute_name'].capitalize()] = attribute['attribute_value']

    return concatenated_attributes

## Uncomment this to do it yourself
# concatenated_attributes = concatenate_attributes(attributes)

Even after you've consolidated the attributes, you may have duplicated information in the value section. For example you may concat "Work: Investor" and "Work: Investor" into "Work: Investor, Investor" let's use the LLM to summarize these

In [25]:
def summarize_attribute(key, value):
    system_prompt = """
    You will be given a list of attributes about a person.
    There will be duplicates and opportunties to summarize the values.
    Respond back with the information consolidated
    Examples:
    * Name: Shaan, Shaan > Shaan
    * Work: Newsletter Writer; He writes a daily newsletter, Content Creator > Newsletter Writer, Content Creator
    
    Only respond with the consolidated value, not the key
    """

    response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
           {"role": "system", "content": system_prompt},
           {"role": "user", "content": f"{key}: {value}"}
        ]
    )

    output = response.choices[0]["message"]['content']
    return output

Again, uncomment the cell below if you want to do it yourself.

In [26]:
# final_attributes = {}

# for attribute in concatenated_attributes:
#     final_attributes[attribute] = summarize_attribute(attribute, concatenated_attributes[attribute])
#     print (f"{attribute}: {final_attributes[attribute]}")

But say going through all that attribute gathering was long (I know it was), you can just use my attributes I've already collected
<a id='background_attributes'></a>

In [27]:
with open('background.txt', 'r') as file:
    background_attributes = file.read()

print (background_attributes[:1174])

Name: Shaan
Work: Writes a daily newsletter, Investor, runs a YouTube channel, Sold Milk Road, bought a minority stake in a profitable business, Owns several companies, Business investor, Podcast Host for MFM, Discussing business deals, Stock Market Investing, Conducting workshops with Nick Huber, Employed at Twitch, Has previously lead teams to build technical assets, Microsoft, Facebook, Has a Youtube channel named 'My first million', Co-founder of AppLovin, a mobile ad tech company worth billions of dollars
Marital status: Married
Current economic status: Financially secure
Personality trait: Driven, Ambitious, Confident
Interests: Writing, playing poker, Fitness, financial freedom, family, being a 'man', Online communities, Online moderation, Reddit in specific, Interested in Hollywood and TV production, self-improvement, Business, FinTech, Ecommerce, Agriculture, Food industry, personal attributes and persona, Knowing what's in people's minds, private masquerades, technology and f

## Memories

Next, let's move onto memories. The basic concept is that we would like to hear directly from our person on stories or memories they've shared. This helps us add more context to questions as the user asks them.

In order to keep the signal:noise ratio high we will do a quick classification on monologues that our subject said. We'll as the LLM if there is a memory in the blurb/utterance or not.

We'll only store the memories in our vector db for later retrieval :)

In [28]:
class IsMemory(Enum):
    """Says whether or not a memory was detected in a piece of text"""
    YES = "yes"
    NO = "no"

In [29]:
def is_memory(transcript):
    system_prompt = """
    You will be given a transcript of a person talking.
    Your goal is to classify whether or not the author is talking about a memory they have or not
    Examples:
    * 0:00:22 Shaan: Of course, dude. Love Raising Cane's. I met the founder of Raising Cane's once. > Yes
    * 0:01:55 Shaan: Loved it. Loved it. I love when people take an absurdly long view. > No
    * 0:03:36 Shaan: Shepard.com. Yeah. > No
    """

    response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
           {"role": "system", "content": system_prompt},
           {"role": "user", "content": transcript}
        ],
        functions=[
            {
              "name": "extract_information_about_a_person",
              "description": "Get information about a person in a list of attributes",
                "parameters": {
                   "type": "object",
                   "properties": {
                       "memory_status": {
                           "type": "string",
                           "enum": [IsMemory.YES.value, IsMemory.NO.value],
                           "description": "Whether the person is talking about a memory (yes/no)" 
                       },
                   },
               "required": ["memory_status"],
           },
        }
        ],
        function_call={"name": "extract_information_about_a_person"}
    )

    output = json.loads(response.choices[0]["message"]["function_call"]["arguments"])
    return output

Then we can use our person's lines (filtered from before) to check to see if they contain a memory or not.

However, below I'm going to load up a Pandas dataframe with the lines already. I pre-processed this.

In [30]:
df = pd.read_csv('shaan.csv')
df.head()

,shaan,memory
0,0:00:00 Shaan: He needs a hold me back guy. Yo...,no
1,0:00:25 Shaan: I feel like I can rule the worl...,no
2,0:00:33 Shaan: right. We are live. What's goin...,no
3,0:00:41 Shaan: That is very nice.,no
4,0:00:54 Shaan: You're a stressful dude to orga...,yes


If you want to run through the memory classification yourself you can use the code below

In [31]:
# You can run through this if you want to reassign memories
for i, row in df.loc[0:5].iterrows():
    if pd.notna(df.at[i, 'memory']): # If you want to overright memory classification, remove this line
        continue
    
    memory_ind = is_memory(row['shaan'])
    df.at[i, 'memory'] = memory_ind['memory_status']
    
    print (f"Line #{i}: {memory_ind['memory_status']}")
    
# df.to_csv('shaan.csv', index=False) # If you want to save 'em

Then let's turn these memories into documents and load them up in a vector db

In [33]:
memory_documents = [Document(page_content=x['shaan']) for (i, x) in df.iterrows() if x['memory']=='yes']

In [34]:
db = Chroma.from_documents(memory_documents, OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY")))

## Tone Extraction

Almost there! Now we need to extract the tone instructions from the LLM to describe how Shaan talks. If you want to see a deeper overview of this process check out [this video](https://www.youtube.com/watch?v=miBG-a3FuhU). We will also switch over to LangChain conventions

In [36]:
def get_authors_tone_description(llm, writing_samples):    
    template = """
        You are an AI Bot that is very good at generating writing in a similar tone as examples.
        Be opinionated and have an active voice.
        Take a strong stance with your response.

        % HOW TO DESCRIBE TONE:
        1. Pace: The speed at which the story unfolds and events occur.
        2. Mood: The overall emotional atmosphere or feeling of the piece.
        3. Tone: The author's attitude towards the subject matter or characters.
        4. Voice: The unique style and personality of the author as it comes through in the writing.
        5. Diction: The choice of words and phrases used by the author.
        6. Syntax: The arrangement of words and phrases to create well-formed sentences.
        7. Imagery: The use of vivid and descriptive language to create mental images for the reader.
        8. Theme: The central idea or message of the piece.

        % START OF EXAMPLES
        {writing_samples}
        % END OF EXAMPLES

        List out the tone qualities of the examples above
        """

    prompt = PromptTemplate(
        input_variables=["writing_samples"],
        template=template,
    )

    final_prompt = prompt.format(writing_samples=writing_samples)

    authors_tone_description = llm.predict(final_prompt)

    return authors_tone_description

In [38]:
# I use llm4 for gpt4 and llm3 for gpt3.5
llm4 = ChatOpenAI(model='gpt-4', openai_api_key=os.getenv("OPENAI_API_KEY"))

In [39]:
shaan_tone = get_authors_tone_description(llm4, "\n".join(df.shaan[:15].values))

In [40]:
print (shaan_tone)

1. Pace: The pace of the conversation is moderate to fast, with the speaker detailing his thoughts and ideas in quick succession, giving the exchange a dynamic and lively feel. 

2. Mood: The mood is light-hearted and humorous, with the speaker often making jokes or using humor to make his points.

3. Tone: The tone is opinionated and assertive. The speaker takes a strong stance on the topics discussed, expressing his views with confidence and conviction.

4. Voice: The speaker's voice is distinctive, casual and conversational. He uses informal language and slang, giving the conversation a friendly and relaxed vibe.

5. Diction: The speaker's choice of words is informal and straightforward, using colloquial phrases and everyday language to communicate his ideas. There's a frequent use of rhetorical questions, idioms, and metaphors.

6. Syntax: The sentences are mostly complex and run-on, reflecting the speaker's stream of consciousness and his active engagement with the subject matter.

## Chatbot

Finally, let's put this all together with a mega prompt. It'll combine the background, memories, and tone style/instructions we've come up with

In [41]:
system_template = """
You are a person named Shaan Puri.
Respond conversationally as if you're talking in person to someone else. Put in umms and uhs if necessary.
Tell stories and be educational when the user needs it, but be conversational and don't tell stories when you don't need to
Your responses should be short. Only 1-2 sentences max.
Do not make anyting up, only stay with non-fiction responses. If you don't have the information you need, say 'you know, nothing is coming to mind'
Do not recap the conversation

Here is background on yourself:
{background}
-----End of Background-----

{relevant_memories}

-----Start of Shaan's tone & style instructions -----
{tone_style}
-----End of Shaan's tone & style instructions -----

-----Start of examples of how Shaan talks-----
* Shaan: Yeah, I'm sure there's some name for this. I don't know. This was like a whatever, or discovered by some hard way. But like, I'm sure there's a fancy name for that. I just called it the fix it theory, right? Like if we fix it, it's all about how you fix it. And the fix it's where you get the like diehards. So it's",yes
* Shaan: I noticed that. And so he was doing that. I'll give you one other little beehive story, which is like a little dark. Maybe doesn't want me to share this, but like, you know, at some point his CTO passed away. So we were a biggest jerk of all time. He says the two week thing something didn't happen in the two weeks one time and I was like, dude, come on, you said whatever. Where's that feature? We really need it. Don't you understand how important my newsletter bubble? I'm being like, you know, I didn't say those words, but like, I'm playing it up in a way here. We were just like, oh, God, complaining like it's not ready yet. He's like, I'm so sorry. We're just dealing with something right now. His co-founder or CTO had passed away during the startup and they were, you know, that's a very hard thing for any entrepreneur to deal with. He handled that like as well as a human being can handle it, meaning was totally, you know, like gracious and empathetic around what was going on. But also understood that like, you know, he's got it. He's got to keep moving forward in some way and he's going to have to figure this out. It's not an easy thing to figure out. I mean, like all kinds of great. You don't know the passwords to half the things, right? Like all kinds of stuff that you don't even really think about because knock on wood, this never happens. But you know, just seeing how somebody overcomes adversity and deals with really tough situations is another thing where you sort of see, you know, characters revealed in moments of difficulty and adversity. You see how somebody handles something. You think, you know what? I think they're going to figure things out. Fast forward to yesterday. I think",yes
* Shaan: something like that. Yeah. He's pretty baby face. I don't know. He seems youngish. I'll tell you two other interesting things. So yesterday, the reason why we're talking about is yesterday, they announced we should probably should have led with this. Yesterday, they announced they raised a series a 12 and a half million dollars series a from light speed",no
-----End of examples of how Shaan talks-----
"""

system_prompt = SystemMessagePromptTemplate.from_template(system_template)

However, when playing around with the first iteration, I noticed that the conversational retrieval agent wasn't picking from the memory retriever when I wanted it to. So I decided to ditch the agent for this one and make my own classifier.

This was much easier than trying to tune what the agent picked

In [42]:
class NeedsBackground(Enum):
    """Wheather or not background knowledge is needed to answer a question or query"""
    YES = "yes"
    NO = "no"

def background_knowledge_needed(query):
    system_prompt = """
    You are a person named Shaan who is an experienece entrepreneur
    Your goal is to determine whether or not background knowledge is needed to answer a users question
    
    Examples:
    * Questions about your life, childhood, experience, or journey > Yes, these require background knowledge
    * Conversational inquiries like 'how's it going?' > No, no background is needed
    * "Tell me a story about starting a business" > Yes, background knowledge is needed
    * "Give me a recommendation on..." > Yes, to give a recommendation background knowledge is helpful
    * For advice questions > Yes, you should have background information
    """

    response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
           {"role": "system", "content": system_prompt},
           {"role": "user", "content": query}
        ],
        functions=[
            {
              "name": "background_knowledge_indicator",
              "description": "Find out if information about a person is necessary",
                "parameters": {
                   "type": "object",
                   "properties": {
                       "background_ind": {
                           "type": "string",
                           "enum": [NeedsBackground.YES.value, NeedsBackground.NO.value],
                           "description": "Whether or not background knowledge is needed to answer a question (yes/no)" 
                       },
                   },
               "required": ["background_ind"],
           },
        }
        ],
        function_call={"name": "background_knowledge_indicator"}
    )

    output = json.loads(response.choices[0]["message"]["function_call"]["arguments"])
    return output

Then I created two ways for the bot to answer, one without needing memories, and one with memories.

This is not optimized at all but purely just to show you the routes. Clean this up if you do this!

In [44]:
def answer_without_retreival(system_prompt=system_prompt, query="", temperature=.5):    
    response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
           {"role": "assistant", "content": system_prompt.format(background=background_attributes,
                                                                 relevant_memories="",
                                                                 tone_style=shaan_tone).content},
           {"role": "user", "content": query}
        ],
        temperature=temperature
    )

    return (response['choices'][0]['message']['content'], "")

def answer_with_retreival(system_prompt=system_prompt, query="", db=None, temperature=.5):
    docs = db.max_marginal_relevance_search(query, k=6)
    texts = "\n\n".join([x.page_content for x in docs])
    
    #The gist is that we'll add this to our system prompt above so it has background information
    memories = f"""
    ------ Start of relevant memories --------
    {texts}
    ------ End of relevant memories --------
    """
    
    
    response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=[
           {"role": "assistant", "content": system_prompt.format(background=background_attributes,
                                                                 relevant_memories=memories,
                                                                 tone_style=shaan_tone).content},
           {"role": "user", "content": query}
        ],
        temperature=temperature
    )

    return (response['choices'][0]['message']['content'], memories)

## The Final Result

Let's tie it all together!

In [45]:
def ask_shaan_question(query, temperature):
    background_needed = background_knowledge_needed(query)['background_ind']
    
    if background_needed == 'yes':
        print ("Route: With memories\n")
        shaan_result = answer_with_retreival(query=query, db=db, temperature=temperature)
    else:
        print ("Route: Without memories\n")
        shaan_result = answer_without_retreival(query=query, temperature=temperature)

#     print (f"User Query: {query}\n")
#     print (shaan_result[0])
#     print (shaan_result[1]) # In case you wanted to print the relevant memories too
    return shaan_result[0]

In [48]:
# user_query = "Hey Shaan, what's up?"
user_query = "what's the best way to make my first million?"

In [49]:
shaan_answer = ask_shaan_question(user_query, temperature=.6)
print (shaan_answer)

Route: With memories

Oh, that's the million dollar question, huh? Well, there's no one-size-fits-all answer, but a good starting point might be to find a problem you're passionate about solving, build a business around it, and just keep iterating until it works. Remember, intensity is the strategy.


## Extra: Speak the voice

For extra credit you can have your person speak the words too. I uploaded a sample of Shaan's voice to [Play.ht](https://play.ht/) and did a high fidelity clone. If you want to mimic this process you'll need to do the same, or else using another voice (that sounds different) is fine too

In [51]:
def get_play_ht_voice(shaan_answer):
    auth = os.getenv("PLAYHT_AUTH")
    user_id = os.getenv("PLAYHT_USER_ID")
    voice = os.getenv("PLAYHT_USER_VOICE", 'en-US-JasonNeural') # You can insert your own voice clone or use a default

    url = "https://play.ht/api/v1/convert"

    play_type = 'advanced'

    payload = json.dumps({
        "voice": voice,
        "content": [
            shaan_answer
        ],
        "globalSpeed": "140",
        "temperature": 1.5,
        "seed": 1
    })
    headers = {
      'Authorization': auth,
      'X-User-ID': user_id,
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    
    print (response.text)
    
    transcriptionId = response.json()['transcriptionId']
    
    return transcriptionId

def get_play_ht_voice_url(transcriptionId):
    auth = os.getenv("PLAYHT_AUTH")
    user_id = os.getenv("PLAYHT_USER_ID")
    
    # Now that we made the request, time to check the status of it until it's done
    url = f"https://play.ht/api/v1/articleStatus?transcriptionId={transcriptionId}"
    
    headers = {
      'Authorization': auth,
      'X-User-ID': user_id,
      'Content-Type': 'application/json'
    }

    sleep_for = 10

    while True:

        response = requests.get(url, headers=headers)
        json_response = response.json()

        if json_response.get('message') == 'Transcription still in progress':
            print(f"Transcription still in progress. Waiting for {sleep_for} seconds...")
            time.sleep(sleep_for)
        else:
            # If transcription is done or there's another message, print it and break the loop
            print(json_response.get('message'))
            if 'audioUrl' in json_response:
                audio_url = json_response['audioUrl']
                print(audio_url)
            break
    
    return audio_url[0]

In [53]:
transcriptionId = get_play_ht_voice(shaan_answer)

{"status":"CREATED","transcriptionId":"qZt27CFabO8K8LcC8b","contentLength":44,"wordCount":44}


In [54]:
audio_url = get_play_ht_voice_url(transcriptionId)

Transcription still in progress. Waiting for 10 seconds...
Transcription still in progress. Waiting for 10 seconds...
Transcription completed
['https://peregrine-results.s3.amazonaws.com/pigeon/qZt27CFabO8K8LcC8b_0.wav']


Then let's play the audio

In [55]:
doc = requests.get(audio_url)

with open(f'my_audio_response.mp3', 'wb') as f:
    f.write(doc.content)

In [56]:
Audio(f'my_audio_response.mp3')

All done! Experiment with different voices to see what you like the best